In [1]:
#导库
import numpy as np
import pandas as pd
import random
import time
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset      # 数据集对象.   数据 -> Tensor -> 数据集 -> 数据加载器
from torch.utils.data import DataLoader         # 数据加载器.
from sklearn.preprocessing import StandardScaler, LabelEncoder

import os

from numpy.ma.extras import median
from unicodedata import category

#更改文件路径
os.chdir('E:\\projects\\Art_learn')
# os.getcwd()

import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'SimHei'  # 替换为你选择的字体
plt.rcParams["axes.unicode_minus"] = False

#获取数据

In [2]:
data = pd.read_csv('./data/rent_forecast/train.csv', encoding='utf-8')
print(data.head())

   ID     位置  出租方式     区  卧室数量  卫的数量  厅的数量  地铁站点  地铁线路   小区名   小区房屋出租数量  居住状态  \
0   0  118.0   NaN  11.0     1     1     1   4.0   2.0  3072  128.90625   NaN   
1   1  100.0   NaN  10.0     1     0     0   5.8   4.0  3152  132.81250   NaN   
2   2  130.0   NaN  12.0     2     2     1   3.7   5.0  5575   42.96875   NaN   
3   3   90.0   NaN   7.0     3     2     2   6.3   2.0  3103   85.93750   NaN   
4   4   31.0   NaN   3.0     2     1     1   NaN   NaN  5182  214.84375   NaN   

         总楼层 房屋朝向        房屋面积  时间  楼层  装修情况         距离      Label  
0  23.636364   东南   86.279378   1   2   NaN  76.416667   5.602716  
1  38.181818    东  170.456802   1   1   NaN  70.916667  16.977929  
2  29.090909   东南  105.925190   1   0   NaN  57.250000   8.998302  
3  58.181818    南  191.989407   1   2   NaN  65.833333   5.602716  
4  54.545455   东北  104.270109   1   0   NaN        NaN   7.300509  


In [3]:
#输出列值
print(data.columns)
print(data.info())

data = data.head(1000)
print(data.shape)

Index(['ID', '位置', '出租方式', '区', '卧室数量', '卫的数量', '厅的数量', '地铁站点', '地铁线路', '小区名',
       '小区房屋出租数量', '居住状态', '总楼层', '房屋朝向', '房屋面积', '时间', '楼层', '装修情况', '距离',
       'Label'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196539 entries, 0 to 196538
Data columns (total 20 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   ID        196539 non-null  int64  
 1   位置        196508 non-null  float64
 2   出租方式      24230 non-null   float64
 3   区         196508 non-null  float64
 4   卧室数量      196539 non-null  int64  
 5   卫的数量      196539 non-null  int64  
 6   厅的数量      196539 non-null  int64  
 7   地铁站点      91778 non-null   float64
 8   地铁线路      91778 non-null   float64
 9   小区名       196539 non-null  int64  
 10  小区房屋出租数量  195538 non-null  float64
 11  居住状态      20138 non-null   float64
 12  总楼层       196539 non-null  float64
 13  房屋朝向      196539 non-null  object 
 14  房屋面积      196539 non-null  float64
 15  时间        

#数据处理

In [4]:
data = data.drop(columns = ['ID','出租方式','地铁站点','小区名','居住状态', '装修情况'])
print(data.columns)
print(data.shape)

Index(['位置', '区', '卧室数量', '卫的数量', '厅的数量', '地铁线路', '小区房屋出租数量', '总楼层', '房屋朝向',
       '房屋面积', '时间', '楼层', '距离', 'Label'],
      dtype='object')
(1000, 14)


In [5]:
miss_count = data.isnull().sum()
print(miss_count)

位置            0
区             0
卧室数量          0
卫的数量          0
厅的数量          0
地铁线路        490
小区房屋出租数量      4
总楼层           0
房屋朝向          0
房屋面积          0
时间            0
楼层            0
距离          490
Label         0
dtype: int64


In [6]:
#对于区，以及位置信息为空的，删除整行
data = data.dropna(subset=['位置', '区'])
print(data.isnull().sum())

位置            0
区             0
卧室数量          0
卫的数量          0
厅的数量          0
地铁线路        490
小区房屋出租数量      4
总楼层           0
房屋朝向          0
房屋面积          0
时间            0
楼层            0
距离          490
Label         0
dtype: int64


#特征工程

In [7]:
#添加新的特征值，表示是否有跌贴
data["是否有地铁"] = data["距离"].notnull().astype(int)

# 用最大距离填充“无地铁”的情况
max_distance = data["距离"].max()
data["距离"] = data["距离"].fillna(max_distance)



In [8]:
data["地铁线路"] = data["地铁线路"].fillna("无地铁")
# One-Hot 编码
data = pd.get_dummies(
    data,
    columns=["地铁线路"],
    prefix="地铁线"
)
print(data.isnull().sum())
print(data.columns)


位置          0
区           0
卧室数量        0
卫的数量        0
厅的数量        0
小区房屋出租数量    4
总楼层         0
房屋朝向        0
房屋面积        0
时间          0
楼层          0
距离          0
Label       0
是否有地铁       0
地铁线_1.0     0
地铁线_2.0     0
地铁线_3.0     0
地铁线_4.0     0
地铁线_5.0     0
地铁线_无地铁     0
dtype: int64
Index(['位置', '区', '卧室数量', '卫的数量', '厅的数量', '小区房屋出租数量', '总楼层', '房屋朝向', '房屋面积',
       '时间', '楼层', '距离', 'Label', '是否有地铁', '地铁线_1.0', '地铁线_2.0', '地铁线_3.0',
       '地铁线_4.0', '地铁线_5.0', '地铁线_无地铁'],
      dtype='object')


In [ ]:
#用平均值填充平均房屋出租数的均值
num_cols = data.select_dtypes(include=np.number).columns
for col in num_cols:
    data[col] = data[col].fillna(data[col].median())

print(data.isnull().sum())

In [ ]:
#将object类别的值替换为数字代替
obj_cols = data.select_dtypes(include=['object']).columns
#存储对应映射
category_dic = {}
for col in obj_cols:
    unique_value = data[col].dropna().unique()

    map_dict = {value: idx for idx, value in enumerate(unique_value)}
    print(map_dict)
    category_dic[col] = map_dict
    data[col] = data[col].map(map_dict)

print(data.head())





In [ ]:
print(data.isnull().sum())
data.shape